### Week 10 Activities

1. Read data samples from `week-10/datasets/Clustering-Datasets-master/02. Synthetic/3MC.csv`. There are 400 two-dimensional (i.e., the dimension being x and y) samples in the csv.

In [ ]:
import pandas as pd

2. Split the datset into training and test (80%/20% split)


3. Use the training set to find 3 clusters of the dataset. **PLEASE DO NOT USE scikit-learn** based clustering libraries. Write your own k-means method implementation.


4. Evaluate clustering performances in task #3 based on **SSE**, **RandIndex**, **Silhouette Coefficient**.


5. If you increase number of clusters in task #3 what type of performance changes do you observe? Please use the elbow technique to determine best number of clustering for the dataset.


6. Just think about implementing hierarchical clustering, producing a dendrogram and apply a cut to obtain the desired number of clusters.
